# 14 Inheritance: For Better or for Worse
Some notes, observations and questions along chapter 14.

- superclasses and subclasses are tightly coupled; meaning that changes to one part of the program may have unexpected and far-reaching effects in other parts

## The super() Function
- used to call the corresponding method of the superclass when when a method of the subclass overrides a method of a superclass

In [3]:
# example

from collections import OrderedDict

class LastUpdatedOrderedDict(OrderedDict):
    """Store items in the order they were last updated"""

    def __setitem__(self, key, value):
        super().__setitem__(key, value) # calls method from superclass
        self.move_to_end(key) # adds its own implementation

In [5]:
# same for __init__

class LastUpdatedOrderedDict(OrderedDict):
    def __init__(self, a, b) :
        super().__init__(a, b)
        ...  # more initialization code

We don't need to pass anything to the `super()` except if we use multiple inheritance and we want to skip over part of the MRO.

## Subclassing Built-In Types Is Tricky
- when we subclass build-in types (written in C), and override a method, mostly the parent's method called with `super()` mostly won't call overridden methods in subclasses:

In [ ]:
class DoppelDict(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2) # calling dict's `__setitem__` method, but with the value appearing twice in a list

dd = DoppelDict(one=1) # we expect the dict to have set key "one" to the value of "2"
dd

{'one': 1}

But the `__init__` method inherited from `dict` clearly ignored that `__setitem__` was overridden: the value of 'one' is not duplicated.

In [ ]:
dd['two'] = 2 # using [] for triggering `__setitem__` directly: this works
dd

{'one': 1, 'two': [2, 2]}

In [ ]:
dd.update(three=3) # the update method from dict does not use our version of `__setitem__` either
dd

{'one': 1, 'two': [2, 2], 'three': 3}

This built-in behavior is a violation of a basic rule of object-oriented programming: **late binding**, meaning in any call of the form `x.method()`, the exact method to be called must be determined at runtime.

- another example:

In [10]:
class AnswerDict(dict):
    def __getitem__(self, key): # `AnswerDict.__getitem__` always returns 42, no matter what the key
        return 42
    
ad = AnswerDict(a='foo')
ad['a'] # returns 42, as expected

42

In [11]:
d = {}
d.update(ad) # d is an instance of plain dict, which we update with ad
d['a'] # the dict.update method ignored our `AnswerDict.__getitem__`

'foo'

Subclassing built-in types like dict or list or str directly is error-prone because the built-in methods mostly ignore user-defined overrides. Instead of subclassing the built-ins, derive your classes from the collections module using UserDict, UserList, and UserString, which are designed to be easily extended.

- inheriting from `collectins.UserDict` instead from `dict`, will solve the above problems:

In [12]:
import collections

class DoppelDict2(collections.UserDict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)

dd = DoppelDict2(one=1)
dd

{'one': [1, 1]}

In [13]:
dd['two'] = 2
dd

{'one': [1, 1], 'two': [2, 2]}

In [14]:
dd.update(three=3)
dd

{'one': [1, 1], 'two': [2, 2], 'three': [3, 3]}

In [15]:
class AnswerDict2(collections.UserDict):
    def __getitem__(self, key):
        return 42
    
ad = AnswerDict2(a='foo')
ad['a']

42

In [16]:
d = {}
d.update(ad)
d['a']

42

In [17]:
d

{'a': 42}

## Multiple Inheritance and Method Resolution Order
- diamond problem when several classes implement the same method in multiple inheritance

In [18]:
class Root: # provides ping, pong, and __repr__
    def ping(self):
        print(f'{self}.ping() in Root')

    def pong(self):
        print(f'{self}.pong() in Root')

    def __repr__(self):
        cls_name = type(self).__name__
        return f'<instance of {cls_name}>'


class A(Root): #  ping and pong methods in class A both call super()
    def ping(self):
        print(f'{self}.ping() in A')
        super().ping()

    def pong(self):
        print(f'{self}.pong() in A')
        super().pong()


class B(Root): # only the ping method in class B calls super()
    def ping(self):
        print(f'{self}.ping() in B')
        super().ping()

    def pong(self):
        print(f'{self}.pong() in B')


class Leaf(A, B): # Leaf implements only ping, and it calls super().
    def ping(self):
        print(f'{self}.ping() in Leaf')
        super().ping()

In [ ]:
leaf1 = Leaf()
leaf1.ping() # leaf1.ping() activates the ping methods in Leaf, A, B, and Root, because the ping methods in the first three classes call super().ping()

<instance of Leaf>.ping() in Leaf
<instance of Leaf>.ping() in A
<instance of Leaf>.ping() in B
<instance of Leaf>.ping() in Root


In [ ]:
leaf1.pong() # leaf1.pong() activates pong in A via inheritance, which then calls super.pong(), activating B.pong.

<instance of Leaf>.pong() in A
<instance of Leaf>.pong() in B


This behavior arises due to Python's Method Resolution Order (MRO): The super() function doesn’t necessarily call the method in the parent class. Instead, it calls the next method in the MRO of the class hierarchy.

- `super()` works based on the MRO, not the inheritance hierarchy directly. The MRO ensures that:

    - Classes are searched in a linear order
    - Each class in the hierarchy is called exactly once

- MRO: determines the activation order
- `super()`: determines whether a particular method will be activated at all

In [21]:
Leaf.__mro__

(__main__.Leaf, __main__.A, __main__.B, __main__.Root, object)

In [28]:
class U(): # class outside of inheritance tree between Root, A, B and Leaf
    def ping(self):
        print(f'{self}.ping() in U')
        super().ping() # What does super().ping() do? Answer: it depends. 

class LeafUA(U, A): 
    def ping(self):
        print(f'{self}.ping() in LeafUA')
        super().ping()

In [29]:
u = U()
u.ping() # error, because object (U's superclass) doesn't implement ping()

<__main__.U object at 0x77ef1842e4e0>.ping() in U


AttributeError: 'super' object has no attribute 'ping'

In [30]:
leaf2 = LeafUA()
leaf2.ping()

<instance of LeafUA>.ping() in LeafUA
<instance of LeafUA>.ping() in U
<instance of LeafUA>.ping() in A
<instance of LeafUA>.ping() in Root


The super().ping() call in LeafUA activates U.ping, which cooperates by calling super().ping() too, activating A.ping, and eventually Root.ping.

In [31]:
LeafUA.__mro__

(__main__.LeafUA, __main__.U, __main__.A, __main__.Root, object)

In [33]:
# but if we had the order of inheritance reversed:
class LeafAU(A, U): 
    def ping(self):
        print(f'{self}.ping() in LeafUA')
        super().ping()

LeafAU.__mro__ #  U gets to be after Root

(__main__.LeafAU, __main__.A, __main__.Root, __main__.U, object)

## Mixin Classes
- designed to be subclassed together with at least one other class in a multiple inheritance arrangement
- adds or customizes the behavior of child class </br>
</br>
- mixins might depend on a sibling class that implements or inherits methods with the same signature
- to make its contribution, a mixin usually needs to appear before other classes in the MRO of a subclass that uses it</br>
</br>
- mixins should never be instantiated
- mixins should provide a single specific behavior, implementing few and very closely related methods
- mixins should avoid keeping any internal state

In [34]:
def _upper(key): # helper function takes a key of any type, and tries to return key.upper()
    try:
        return key.upper()
    except AttributeError:
        return key

class UpperCaseMixin: # class designed to provide case-insensitive access to mappings
    def __setitem__(self, key, item):
        super().__setitem__(_upper(key), item)

    def __getitem__(self, key):
        return super().__getitem__(_upper(key))

    def get(self, key, default=None):
        return super().get(_upper(key), default)

    def __contains__(self, key):
        return super().__contains__(_upper(key))

In [36]:
# UpperDict needs no implementation of its own, but UpperCaseMixin must be the first base class, otherwise the methods from UserDict would be called instead
class UpperDict(UpperCaseMixin, collections.UserDict): 
    pass

d = UpperDict([('a', 'letter A'), (2, 'digit two')])
list(d.keys())

['A', 2]

In [37]:
d['b'] = 'letter B'
'b' in d

True

In [38]:
d['a'], d.get('B')

('letter A', 'letter B')

In [39]:
list(d.keys())

['A', 2, 'B']

## Multiple Inheritance in the Real World
- ABCs can be used as mixins too
- rest of this section was presenting examples os multiple inheritance and mixins from several python libraries like tkinter

## Coping with Inheritance
- favor object composition and delegation over class inheritance
    - distinguishing what classes are supposed to *be* and what they are supposed to *do*

- remember that inheritance is a "is a" relationship

- interface inheritance should use ABCs if possible

- if a class is designed to provide method implementations for reuse by multiple unrelated subclasses, without implying an “is-a” relationship, it should be an explicit mixin class

- not all classes are designed to be extended and inheriting from a class that overrides methods from its parent class might be error prone
    - their methods might depend on an internal state

- some say "all non-leaf classes should be abstract"
    - at least, when we have to create a mixin class with some constraint functionality, then we have to make it explicit on the naming of the mixin classes